In [30]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision.datasets
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

num_epochs=50
learning_rate=0.001
batch_size=128

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

train_dataset=torchvision.datasets.CIFAR10(root='./data',train=True,transform=transform,download=True)
test_dataset=torchvision.datasets.CIFAR10(root='./data',train=False,transform=transform,download=True)


Files already downloaded and verified
Files already downloaded and verified


In [31]:
print(len(train_dataset))
print(len(test_dataset))

50000
10000


In [32]:
from torch.utils.data import random_split

train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=1, shuffle=False)



In [33]:
train_dataset[0][0].shape

torch.Size([3, 32, 32])

In [34]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc1 = nn.Linear(4 * 4 * 128, 1000)
        self.fc2 = nn.Linear(1000, 512)
        self.fc3 = nn.Linear(512, 100)
        self.fc4 = nn.Linear(100, 10)
        self.drop_out = nn.Dropout(p=0.5)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.fc4(out)
        return out

learning_rate = 0.001
num_epochs = 50

model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=5, verbose=True)

total_step = len(train_loader)
best_val_accuracy = 0.0
patience = 5
counter = 0
for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Loss: {loss.item()}')

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        val_accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{num_epochs}] - Validation Accuracy: {val_accuracy:.2f}%')

        scheduler.step(val_accuracy)

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), 'best_convnet_model.pth')
            counter = 0
        else:
            counter += 1
        if counter >= patience:
            print("Early stopping. No improvement in validation accuracy.")
            break

model.load_state_dict(torch.load('best_convnet_model.pth'))
model.eval()

test_correct = 0
test_total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()
    test_accuracy = 100 * test_correct / test_total
    print(f'Test Accuracy: {test_accuracy:.2f}%')







Epoch [1/50], Step [100/313], Loss: 1.5179927349090576
Epoch [1/50], Step [200/313], Loss: 1.4036657810211182
Epoch [1/50], Step [300/313], Loss: 1.245557188987732
Epoch [1/50] - Validation Accuracy: 58.32%
Epoch [2/50], Step [100/313], Loss: 1.2021178007125854
Epoch [2/50], Step [200/313], Loss: 1.098151445388794
Epoch [2/50], Step [300/313], Loss: 0.7357503175735474
Epoch [2/50] - Validation Accuracy: 60.32%
Epoch [3/50], Step [100/313], Loss: 0.7284601330757141
Epoch [3/50], Step [200/313], Loss: 0.911190390586853
Epoch [3/50], Step [300/313], Loss: 0.981538712978363
Epoch [3/50] - Validation Accuracy: 67.01%
Epoch [4/50], Step [100/313], Loss: 0.6408079266548157
Epoch [4/50], Step [200/313], Loss: 0.7199646830558777
Epoch [4/50], Step [300/313], Loss: 0.6706714630126953
Epoch [4/50] - Validation Accuracy: 67.71%
Epoch [5/50], Step [100/313], Loss: 0.7189332246780396
Epoch [5/50], Step [200/313], Loss: 0.665956437587738
Epoch [5/50], Step [300/313], Loss: 0.6318879723548889
Epoch [5

In [36]:
from random import randint

labels = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]

for _ in range(5):
    random_index = randint(1,10000)
    image, label = test_dataset[random_index]

    model.eval()
    with torch.no_grad():
        predictions = model(image.unsqueeze(0))

    _, predicted = torch.max(predictions, 1)

    print("True:", labels[label], "Predicted:", labels[predicted])



True: deer Predicted: deer
True: ship Predicted: truck
True: truck Predicted: truck
True: airplane Predicted: airplane
True: truck Predicted: truck
